In [1]:
# keras & tensorflow
from keras.models import load_model

# File
import json
import pickle

# DIP
from PIL import Image
from PIL.Image import fromarray
import cv2

# numpy
from numpy import asarray
from numpy import expand_dims
from numpy import round
from numpy import linalg

# os
from os import listdir, mkdir
from os.path import exists
from os import system

# date & time
from datetime import datetime
import time

# GUI
from tkinter import *
from tkinter import messagebox

In [3]:
signature_path = "D:\\College\\Semester 8\\Coding\\Data\\signature.pkl"
if exists(signature_path):
    mySignature = open("D:\\College\\Semester 8\\Coding\\Data\\signature.pkl", "rb")
    faceDatabase = pickle.load(mySignature)
    mySignature.close()
else:
    faceDatabase = {}
    
# HaarCascade
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# FaceNet
MyFaceNet = load_model("D:\\College\\Semester 8\\Coding\\Models\\FaceNet\\facenet_keras.h5")

absensi_path = "D:\\College\\Semester 8\\Coding\\Data\\absensi.json"
mahasiswa_path = "D:\\College\\Semester 8\\Coding\\Data\\mahasiswa.json"

In [6]:
def absen(nobp):
    signatureBase = faceDatabase
    status = 0
    for key, value in signatureBase.items():
        if key == nobp:
            status = 1
            
            cap = cv2.VideoCapture(0)
            # cap = cv2.VideoCapture(1)
            t=1
            while t:
                # time.sleep(0)
                _, imgVideo = cap.read()
                FaceDetect = HaarCascade.detectMultiScale(imgVideo, 1.3, 10)
                
                for (x1, y1, width, height) in FaceDetect:
                    x1, y1 = abs(x1), abs(y1)
                    x2, y2 = x1 + width, y1 + height
                    
                    img = cv2.cvtColor(imgVideo, cv2.COLOR_BGR2RGB)
                    img = fromarray(img)
                    img_array = asarray(img)
                    
                    face = img_array[y1:y2, x1:x2]
                    
                    face = fromarray(face)
                    face = face.resize((160,160))
                    wajah = face
                    face = asarray(face)
                    
                    face = face.astype('float32')
                    mean, std = face.mean(), face.std()
                    face = (face - mean) / std
                    
                    face = expand_dims(face, axis=0)
                    signature = MyFaceNet.predict(face)
                    
                    distance = linalg.norm(value - signature)
                    if (distance > 7):
                        identity = "Unknown"
                        cv2.rectangle(imgVideo, (x1,y1), (x2,y2), (0,0,255), 2)
                        cv2.rectangle(imgVideo, (x1,y1-40), (x2,y1), (0,0,255), -2)
                        cv2.putText(imgVideo, identity, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
                    else:
                        identity = key
                        cv2.rectangle(imgVideo, (x1,y1), (x2,y2), (0,255,0), 2)
                        cv2.rectangle(imgVideo, (x1,y1-40), (x2,y1), (0,255,0), -2)
                        cv2.putText(imgVideo, identity + ", " + str(round(distance, 2)), (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
                        
                cv2.imshow('Face Recognition', imgVideo)   
                
                # # Jika ingin proses verifikasi berjalan otomatis
                # time.sleep(5)
                # break
                # Menampilkan proses verifiaksi di kamera
                k = cv2.waitKey(5) & 0xFF
                if k == 27:
                    t-=1
            cv2.destroyAllWindows()
            cap.release()
            
    if status != 1:
        messagebox.showwarning("Rekam Absensi", "Data anda tidak ditemukan.\nSilahkan lakukan training wajah terlebih dahulu")
        
    if identity == "Unknown":
        messagebox.showwarning("Rekam Absensi", "Wajah tidak cocok. Silahkan isi password jika nobp identitas anda benar")
    else:
        # check if absensi.json and mahasiswa.json is ready
        if absensi_path:
            print("absensi.json is ready")
        if mahasiswa_path:
            print("mahasiswa.json is ready")
            
        # takeAttendance(nobp)
        messagebox.showinfo("Rekam Absensi", "Absensi anda untuk hari ini berhasil direkam")
        
absen("18101152630069")